In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cookies = {
    'cookiesession1': '678B28F695C90B262BFC00B79BBD6CC1',
    'lhc_per': 'vid|a3d85efe963899fee2b5',
    '_gid': 'GA1.3.289911229.1685154797',
    '_ga': 'GA1.1.2075372866.1684772919',
    '_ga_3C534XGNC2': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_Q33BVTTDL1': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_QHEB12PC6D': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'cookiesession1=678B28F695C90B262BFC00B79BBD6CC1; lhc_per=vid|a3d85efe963899fee2b5; _gid=GA1.3.289911229.1685154797; _ga=GA1.1.2075372866.1684772919; _ga_3C534XGNC2=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_Q33BVTTDL1=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_QHEB12PC6D=GS1.1.1685154797.4.1.1685154882.0.0.0',
    'Pragma': 'no-cache',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [4]:
def download(url, file_name, retry = 3):
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 360, cookies=cookies, headers=headers)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e)
                time.sleep(0.5)

In [5]:
files = glob('pages-unimas/*.json')
len(files)

1067

In [6]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [7]:
filtered_pdfs = []
for p in tqdm(pdfs):
#     if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
#         continue
#     if not p.endswith('.pdf'):
#         continue
#     if 'ade.sagepub.com' in p:
#         continue
    
#     if p.startswith('/jspui'):
#         p = 'https://oarep.usim.edu.my' + p
    
    if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
        continue
    if not p.endswith('.pdf'):
        continue
        
    if not p.startswith('http'):
        continue
        
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|█████████████████████████████████| 17007/17007 [00:00<00:00, 515561.17it/s]


16986

In [8]:
# filtered_pdfs

In [9]:
unimas = filtered_pdfs

In [10]:
# !rm -rf usim
!mkdir unimas

In [11]:
max_worker = 1

unimas = sorted(list(unimas))
for i in tqdm(range(0, len(unimas), max_worker)):
    urls = [(url, os.path.join('unimas', f'{i}-{no}.pdf')) for no, url in enumerate(unimas[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

  0%|                                      | 3/16986 [00:07<12:04:49,  2.56s/it]

0 HTTPConnectionPool(host='annalseas.faa.ro', port=80): Max retries exceeded with url: /download/4%20Abang%20Ekhsan%20.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc3c899db50>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPConnectionPool(host='annalseas.faa.ro', port=80): Max retries exceeded with url: /download/4%20Abang%20Ekhsan%20.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc3c899db50>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='annalseas.faa.ro', port=80): Max retries exceeded with url: /download/4%20Abang%20Ekhsan%20.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc3c899d790>: Failed to establish a new connection: [Errno -2] Name or service not known'))


  0%|                                      | 4/16986 [00:10<11:40:22,  2.47s/it]

0 HTTPConnectionPool(host='annalseas.faa.ro', port=80): Max retries exceeded with url: /download/5%20Abang%20Ekhsan%20.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc3c899db80>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPConnectionPool(host='annalseas.faa.ro', port=80): Max retries exceeded with url: /download/5%20Abang%20Ekhsan%20.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc3c899db80>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='annalseas.faa.ro', port=80): Max retries exceeded with url: /download/5%20Abang%20Ekhsan%20.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc3c899d7c0>: Failed to establish a new connection: [Errno -2] Name or service not known'))


  1%|▍                                    | 224/16986 [06:12<8:37:49,  1.85s/it]

...

100%|████████████████████████████████▉| 16973/16986 [10:43:53<25:26, 117.43s/it]

0 HTTPSConnectionPool(host='ir.kochi-u.ac.jp', port=443): Max retries exceeded with url: /dspace/bitstream/10126/5426/2/8-1.71.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc3c8826c10>: Failed to establish a new connection: [Errno 110] Connection timed out'))
1 HTTPSConnectionPool(host='ir.kochi-u.ac.jp', port=443): Max retries exceeded with url: /dspace/bitstream/10126/5426/2/8-1.71.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc3abffe070>: Failed to establish a new connection: [Errno 110] Connection timed out'))
2 HTTPSConnectionPool(host='ir.kochi-u.ac.jp', port=443): Max retries exceeded with url: /dspace/bitstream/10126/5426/2/8-1.71.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc3abfe34f0>: Failed to establish a new connection: [Errno 110] Connection timed out'))


100%|█████████████████████████████████▉| 16977/16986 [10:50:33<10:35, 70.59s/it]

0 HTTPSConnectionPool(host='my.laureate.net', port=443): Max retries exceeded with url: /Faculty/docs/Faculty%20Documents/INTI%20Conferences/Parallel%20Sessions%201/1B/1B-02-P52%20(Malaysia).pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc3c88f9130>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
1 HTTPSConnectionPool(host='my.laureate.net', port=443): Max retries exceeded with url: /Faculty/docs/Faculty%20Documents/INTI%20Conferences/Parallel%20Sessions%201/1B/1B-02-P52%20(Malaysia).pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc3c8903250>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
2 HTTPSConnectionPool(host='my.laureate.net', port=443): Max retries exceeded with url: /Faculty/docs/Faculty%20Documents/INTI%20Conferences/Parallel%20Sessions%201/1B/1B-02-P52%20(Malaysia).pdf (Caused by NewConnectionError('<urllib3.connection.

100%|██████████████████████████████████| 16986/16986 [10:50:45<00:00,  2.30s/it]


In [12]:
!du -hs unimas

68G	unimas
